# Augmenting prompts
Using the `{variable}` syntax, we can augment prompts to "%bob". This allows providing necessary information to have code that fits best to requested tasks. This notebook demonstrates how to use this, e.g. to let bob come up with regular expressions for retrieving information from filenames.

In [1]:
filenames = ["exp01_c1_t0001.tif", "exp01_c2_t0001.tif", "exp01_c1_t0002.tif", "exp01_c2_t0002.tif"]

In [2]:
import bia_bob

In [3]:
filenames

['exp01_c1_t0001.tif',
 'exp01_c2_t0001.tif',
 'exp01_c1_t0002.tif',
 'exp01_c2_t0002.tif']

In [4]:
%%bob
I have the following list of filenames stored in variable "filenames":

{filenames}

Please write Python code using regular expressions for extracting
the time point t and the channel c  as numbers from the filenames. 
Save them in two lists t_list and c_list.
Print out both lists.

In [5]:
import re

# Initialize lists to store channels and time points
t_list = []
c_list = []

# Loop through filenames and extract channel and time point
for filename in filenames:
    match = re.search(r'_c(\d+)_t(\d+)', filename)
    if match:
        c_list.append(int(match.group(1)))
        t_list.append(int(match.group(2)))

# Print out the lists
print("t_list:", t_list)
print("c_list:", c_list)

t_list: [1, 1, 2, 2]
c_list: [1, 2, 1, 2]


## Computing image object features
The same approach can be used for computing additional features after feature extraction from images. We just provide the existing `columns` to Bob and ask it to compute a given feature from them.

In [6]:
%%bob
Load blobs.tif from disk. Segment the bright objects 
using Otsu's method and connected component labeling. 
Extract shape descriptors such as minor/major axis from 
the label image using scikit-image regionprops_table. 
Store the measurements in a pandas DataFrame

In [7]:
import pandas as pd
from skimage.io import imread
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops_table

# Load the image
image = imread('blobs.tif')

# Apply Otsu's thresholding
thresh = threshold_otsu(image)
binary = image > thresh

# Perform connected component labeling
label_image = label(binary)

# Extract shape descriptors
properties = ['label', 'minor_axis_length', 'major_axis_length']
measurements = regionprops_table(label_image, properties=properties)

# Store the measurements in a pandas DataFrame
df = pd.DataFrame(measurements)
df.head()

,label,minor_axis_length,major_axis_length
0,1,16.819060,34.957399
1,2,11.803854,21.061417
2,3,28.278264,30.212552
3,4,23.064079,24.535398
4,5,19.833058,31.162612


In [8]:
columns = df.columns
columns

Index(['label', 'minor_axis_length', 'major_axis_length'], dtype='object')

In [9]:
%%bob
I have a DataFrame `df` with the following columns: {columns}
How can I put a new column `aspect_ratio` into the table?

In [10]:
import numpy as np

df['aspect_ratio'] = df['minor_axis_length'] / df['major_axis_length']
df.head()

,label,minor_axis_length,major_axis_length,aspect_ratio
0,1,16.819060,34.957399,0.481130
1,2,11.803854,21.061417,0.560449
2,3,28.278264,30.212552,0.935977
3,4,23.064079,24.535398,0.940033
4,5,19.833058,31.162612,0.636438
